<a href="https://colab.research.google.com/github/datawilly/Simple_chatbot/blob/main/Copia_de_Simple_Chatbot_con_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot simple con implementacion de APIs

En esta celda, importamos varias librerías esenciales para la creación y mejora de un chatbot y para el análisis de texto. Utilizamos **nltk.chat** para crear un chatbot con reglas simples de coincidencia de patrones y **reflections** para manejar respuestas comunes. Importamos **CountVectorizer** de **scikit-learn** para convertir texto en vectores numéricos, lo cual es útil para procesar y clasificar el texto, y **MultinomialNB** para construir un modelo Naive Bayes que prediga intenciones basadas en esos vectores.

Además, utilizamos **requests** para hacer solicitudes HTTP, como llamadas a APIs de clima o noticias. **textwrap** nos ayuda a formatear el texto en líneas más legibles dentro de la consola, y **re** permite el uso de expresiones regulares para encontrar patrones en el texto. También importamos **json** para almacenar interacciones del chatbot en archivos y para manejar datos en formato JSON de las API. Esta celda configura todo el entorno necesario para manejar el análisis de texto, la interacción con APIs externas y la implementación del chatbot.

In [1]:
from nltk.chat.util import Chat, reflections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import requests
import json
import textwrap
import re
import json

En esta celda, definimos una lista llamada `pairs` que contiene varios patrones de conversación junto con sus respuestas correspondientes. Estos patrones siguen la estructura de expresiones regulares para que el chatbot pueda reconocer diferentes frases y responder de manera apropiada.

Por ejemplo, si el usuario menciona su nombre ("mi nombre es..."), el chatbot responderá con un saludo personalizado. Hay patrones para detectar cuándo el usuario pregunta por el nombre del chatbot, cuándo ofrece disculpas, o cuando simplemente saluda ("hola", "hello").

Además, se incluyen respuestas predefinidas para cuando el usuario pregunta sobre las capacidades del chatbot, su ubicación, o si está lloviendo en algún lugar. También hay una regla para finalizar la conversación cuando el usuario escribe "quit". Este conjunto de patrones es clave para definir el flujo de la conversación y cómo el chatbot interactúa con los usuarios en diferentes situaciones.

In [2]:
pairs = [
         [
          r"(.*) mi nombre es (.*)",
          ["Hola %2, Como estas hoy?",]
         ],
         [
          r"(.*) ayudar (.*) ",
          ["Claro que si, dime que necesitas" , "Yo puedo ayudarte, cuentame",]
         ],
         [
           r"(.*) tu nombre?",
           ["Eso es complicado pero tu puedes solo llamarme robot.",]
         ],
         [
           r"Como estas (.*) ?",
           ["Estoy genial!" , "Mucho mejor ahora que lo preguntas",]
          ],
          [
           r"Lo siento (.*)",
           ["No te preocupes todo esta bien", "Somos humanos, bueno yo no pero te disculpo"]
          ],
          [
           r"Estoy (.*) (bien|genial|contento|feliz)",
           ["Me alegra escucharlo !",]
          ],
          [
           r"(hey|hello|hola|holla)(.*)",
           ["Hola", "Como estas",]
          ],
          [
           r"(.*) que puedes hacer (.*) ?",
           ["Estoy programado para resolver tus dudas",]

          ],
          [
            r"(.*) creo (.*)?",
            ["William Rosales me hizo usando la libreria NLTK ","top secret ;)",]
          ],
          [
            r"(.*) (ubicacion|ciudad|lugar|pais) ?",
            ['Quito, Ecuador',]
          ],
          [
            r"(.*) llueva (.*)",
            ["Ultimamente esta lloviendo muy amenudno en en %2","En %2 hay un gran porcentaje de lluvia",]
          ],
          [
            r"quit",
            ["Adios por ahora, nos vemos pronto :) ","Me gustó hablar contigo, adios :)",]
          ]
]

En esta celda, creamos un objeto `Chat` utilizando la clase `Chat` de la librería **nltk.chat.util**. Este objeto recibe dos argumentos: `pairs` y `reflections`.

- `pairs`: Es la lista de patrones y respuestas predefinidas que hemos configurado en la celda anterior. Define las reglas de interacción del chatbot.
- `reflections`: Es un diccionario incorporado en **nltk** que contiene reglas simples para reflejar palabras, como convertir "yo" en "tú" o "soy" en "eres", lo que permite una conversación más natural.

Este objeto `Chat` es el núcleo del chatbot, ya que manejará las entradas del usuario y proporcionará las respuestas apropiadas basadas en los patrones definidos.

In [3]:
chat = Chat(pairs,reflections)

En esta celda, ejecutamos el método `converse()` del objeto `Chat`, que inicia la conversación con el usuario. Este método pone al chatbot en modo interactivo, esperando la entrada del usuario y respondiendo según los patrones definidos en la lista `pairs`.

Cada vez que el usuario ingresa una frase, el método `converse()` busca el patrón correspondiente en `pairs` y responde con la respuesta adecuada. Si no encuentra un patrón que coincida, utiliza las reglas de reflexiones para generar una respuesta lo más natural posible.

Este es el comando que activa y mantiene el ciclo de conversación del chatbot, permitiendo que el usuario y el chatbot interactúen continuamente hasta que el usuario decida finalizar la conversación.

In [4]:
chat.converse()

>hola
Como estas
>como de llamas
None
>salir
None
>quit
Adios por ahora, nos vemos pronto :) 


Este chatbot, aunque funcional, es bastante básico en cuanto a su estructura y capacidades. Actualmente, se basa en coincidencias simples de patrones predefinidos utilizando expresiones regulares. Si bien puede responder a ciertas preguntas comunes, como su nombre, no tiene la capacidad de manejar preguntas más complejas o adaptarse dinámicamente a nuevos temas de conversación.

Por ejemplo, en la interacción anterior:
- **Usuario**: "Como te llamas"
- **Chatbot**: "None" (sin respuesta clara)

Aquí podemos ver una limitación del sistema, ya que no siempre encuentra la mejor respuesta a frases que varían levemente de las que están predefinidas.

Una forma de mejorar el chatbot es integrar **APIs externas** que permitan expandir sus capacidades, como:
- APIs de clima para proporcionar información meteorológica en tiempo real.
- APIs de noticias para responder preguntas sobre eventos actuales.
- APIs de búsqueda o información general (como Wikipedia o Google) para contestar una mayor variedad de preguntas.

Al integrar APIs, el chatbot podría salir de la estructura rígida de patrones predefinidos y ofrecer respuestas más útiles y variadas, lo que lo haría mucho más interactivo y robusto.


En esta celda, se implementa un modelo de clasificación **Naive Bayes** para predecir las intenciones del usuario a partir de frases de entrada. El flujo del código es el siguiente:

1. **Entrenamiento con frases simples**: Se define un conjunto de frases de entrenamiento en la lista `frases_entrenamiento`, que representan ejemplos básicos de preguntas que el chatbot puede recibir, como "¿Cómo está el clima?" o "Dame las últimas noticias". Paralelamente, se define la lista `intenciones_entrenamiento` que contiene las intenciones correspondientes a cada frase, como "clima", "noticias", "nombre", "agradecer", y "salir".

2. **Vectorización de texto**: Se utiliza el método `CountVectorizer` de **scikit-learn** para convertir las frases de entrenamiento en una matriz de características numéricas, donde cada columna representa una palabra y cada fila representa una frase. Este proceso transforma el texto en un formato comprensible para el modelo de clasificación.

3. **Entrenamiento del modelo**: Se entrena un clasificador **Naive Bayes Multinomial** (`MultinomialNB`), usando las características vectorizadas (`X`) y las intenciones correspondientes (`y`). Este modelo será capaz de predecir la intención del usuario dada una nueva frase.

4. **Función para predecir intenciones**: La función `predecir_intencion(frase)` toma una frase de entrada del usuario, la vectoriza usando el mismo `vectorizer` entrenado, y utiliza el modelo Naive Bayes (`clf`) para predecir la intención correspondiente, devolviendo la intención más probable.

Este proceso permite que el chatbot interprete las preguntas del usuario y determine la acción apropiada, como proporcionar el clima o las últimas noticias.

In [5]:
frases_entrenamiento = [
    "¿Cómo está el clima?", "Dame las últimas noticias",
    "Quiero saber tu nombre", "gracias", "salir"
]
intenciones_entrenamiento = [
    "clima", "noticias", "nombre", "agradecer", "salir"
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(frases_entrenamiento)
y = intenciones_entrenamiento

clf = MultinomialNB()
clf.fit(X, y)

def predecir_intencion(frase):
    X_nueva = vectorizer.transform([frase])
    return clf.predict(X_nueva)[0]

En esta celda, se configuran las **variables de entorno** que contienen las claves de las diferentes APIs utilizadas por el chatbot.

1. **Definición de las variables de entorno**:
   - Se almacenan las claves API necesarias para conectar el chatbot con las siguientes APIs:
     - **OpenWeather**: Utilizada para obtener información meteorológica.
     - **NewsAPI**: Usada para obtener las últimas noticias de acuerdo con una consulta.
     - **Google Custom Search API**: Utilizada para realizar búsquedas en Google.
     - **CSE_ID**: El ID del motor de búsqueda personalizado de Google.
   
2. **Almacenamiento seguro**:
   - Estas claves se almacenan temporalmente en el entorno de ejecución, de modo que no se incluyen en el archivo del notebook. Este enfoque asegura que las claves no se expongan públicamente, protegiendo el acceso a las APIs.


Esta celda implementa el chatbot mejorado que interactúa con los usuarios, proporcionando información sobre el clima, noticias, su nombre, entre otras funcionalidades. Aquí se detallan las partes principales del código:

1. **Diccionario `ciudades_colombia`**:
   - Se define un diccionario con las coordenadas (latitud y longitud) de las principales ciudades de Colombia. Este diccionario se utiliza para obtener el clima de una ciudad específica al hacer consultas a la API de OpenWeather.

2. **Función `obtener_clima(ciudad)`**:
   - Esta función utiliza la **API de OpenWeather** para obtener el clima actual en una ciudad especificada por el usuario. Primero verifica si la ciudad está en el diccionario `ciudades_colombia`. Si es así, hace una llamada a la API usando las coordenadas de la ciudad. La respuesta incluye la temperatura y la descripción del clima (por ejemplo, "nublado"). Se maneja cualquier error de conexión o HTTP con bloques `try-except` para devolver un mensaje de error claro en caso de que falle la solicitud.

3. **Función `guardar_interaccion(usuario, bot)`**:
   - Cada interacción entre el usuario y el chatbot se guarda en un archivo JSON llamado `interacciones.json`. Esto permite almacenar un registro de las conversaciones. La interacción incluye tanto la entrada del usuario como la respuesta del chatbot.

4. **Función `predecir_intencion(frase)`**:
   - Esta función utiliza un modelo de clasificación entrenado previamente para predecir la intención del usuario basándose en la frase de entrada. La frase del usuario es vectorizada y luego el modelo predice si la intención es preguntar sobre el clima, noticias, o si es una despedida, entre otras posibles intenciones.

5. **Función principal `chatbot_mejorado()`**:
   - Es el núcleo de la interacción del chatbot. Esta función ejecuta un bucle donde el chatbot espera la entrada del usuario, predice su intención y responde de acuerdo con las intenciones predefinidas:
     - Si la intención es "clima", solicita la ciudad y llama a `obtener_clima()`.
     - Si es "noticias", (aquí implementaremos la API de noticias).
     - Si es "nombre", responde con su nombre.
     - Si es "agradecer", devuelve un mensaje de cortesía.
     - Si el usuario escribe "salir", el chatbot se despide y el programa finaliza.
   - Las interacciones se guardan después de cada respuesta, permitiendo un registro de la conversación.

Este chatbot es capaz de interpretar frases, proporcionar información dinámica (como el clima), y mantener una conversación básica, mejorada mediante el uso de **APIs externas** y un **modelo de clasificación**.


In [7]:
ciudades_colombia = {
    'bogota': {'lat': 4.7110, 'lon': -74.0721},
    'medellin': {'lat': 6.2442, 'lon': -75.5812},
    'cali': {'lat': 3.4516, 'lon': -76.53199},
    'barranquilla': {'lat': 10.9685, 'lon': -74.7813},
    'cartagena': {'lat': 10.3910, 'lon': -75.4794},
    'cucuta': {'lat': 7.8932, 'lon': -72.5012},
    'pasto': {'lat': 1.2132, 'lon': -77.2811},
    'manizales': {'lat': 5.0700, 'lon': -75.5174},
    'pereira': {'lat': 4.8133, 'lon': -75.6961},
    'ibague': {'lat': 4.4447, 'lon': -75.2424},
    'villavicencio': {'lat': 4.1448, 'lon': -73.6263},
    'bucaramanga': {'lat': 7.1254, 'lon': -73.1194},
    'armenia': {'lat': 4.5297, 'lon': -75.6811},
    'soledad': {'lat': 10.9833, 'lon': -74.7833},
    'sincelejo': {'lat': 9.3043, 'lon': -75.3921},
    'valledupar': {'lat': 10.4631, 'lon': -73.2537},
    'monteria': {'lat': 8.7550, 'lon': -75.8803},
    'neiva': {'lat': 2.9314, 'lon': -75.2803},
    'popayan': {'lat': 2.4479, 'lon': -76.6068},
    'envigado': {'lat': 6.1658, 'lon': -75.5867}
}

def obtener_clima(ciudad):
    api_key = os.getenv('OPENWEATHER_API_KEY')
    ciudad_lower = ciudad.lower().strip()

    if ciudad_lower in ciudades_colombia:
        lat = ciudades_colombia[ciudad_lower]['lat']
        lon = ciudades_colombia[ciudad_lower]['lon']
    else:
        return "No tengo información sobre esa ciudad. Por favor, verifica el nombre o elige una ciudad de Colombia que esté en la lista."

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=es"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if data.get("cod") == 200:
            temp = data['main']['temp']
            descripcion = data['weather'][0]['description']
            return f"El clima en {ciudad.title()} es {descripcion} con una temperatura de {temp}°C."
        else:
            return f"No pude encontrar el clima para {ciudad.title()}. Código de error: {data.get('cod')}, Mensaje: {data.get('message')}"

    except requests.exceptions.HTTPError as errh:
        return f"Error de HTTP: {errh}"
    except requests.exceptions.ConnectionError as errc:
        return "Error de conexión. Verifica tu conexión a internet."
    except requests.exceptions.Timeout as errt:
        return "La solicitud al servidor ha superado el tiempo de espera."
    except requests.exceptions.RequestException as err:
        return f"Error desconocido: {err}"

def guardar_interaccion(usuario, bot):
    data = {
        "usuario": usuario,
        "bot": bot
    }
    with open('interacciones.json', 'a') as archivo:
        json.dump(data, archivo)
        archivo.write('\n')

def predecir_intencion(frase):
    X_nueva = vectorizer.transform([frase])
    return clf.predict(X_nueva)[0]

def chatbot_mejorado():
    print("¡Hola! Soy un chatbot mejorado. Escribe 'salir' para salir.")

    while True:
        usuario = input("Tú: ").lower().strip()
        intencion = predecir_intencion(usuario)

        if intencion == "clima":
            ciudad = input("¿En qué ciudad de Colombia te gustaría saber el clima? ").strip()
            respuesta = obtener_clima(ciudad)
        elif intencion == "noticias":
            respuesta = "\n".join(obtener_noticias())
        elif intencion == "nombre":
            respuesta = "Puedes llamarme 'bot'."
        elif intencion == "agradecer":
            respuesta = "¡De nada! Si tienes más preguntas, estaré aquí."
        elif intencion == "salir":
            print("Adiós por ahora. ¡Nos vemos pronto!")
            break
        else:
            respuesta = "No entendí eso. ¿Puedes reformularlo?"

        print("Chatbot:", respuesta)

        guardar_interaccion(usuario, respuesta)

chatbot_mejorado()

¡Hola! Soy un chatbot mejorado. Escribe 'salir' para salir.
Tú: clima
¿En qué ciudad de Colombia te gustaría saber el clima? pasto
Chatbot: El clima en Pasto es lluvia ligera con una temperatura de 21.13°C.
Tú: salir
Adiós por ahora. ¡Nos vemos pronto!


En esta celda actualizamos el código anterior incluyendo la API de consulta de noticias.

1. **Función `obtener_noticias(tema)`**:
   - Esta función utiliza la **NewsAPI** para obtener las últimas noticias relacionadas con un tema proporcionado por el usuario. Se limita a mostrar las primeras 5 noticias con el título y la descripción.
   - También se manejan los errores de conexión o de la API mediante un bloque `try-except`.

2. **Función `predecir_intencion(frase)`**:
   - Utiliza expresiones regulares para identificar la intención del usuario en función de las palabras clave en su frase. Las intenciones reconocidas incluyen preguntar por el clima, noticias, el nombre del chatbot, o dar las gracias.
   - Si no se reconoce ninguna intención, la función devuelve `None`.

3. **Función principal `chatbot_mejorado()`**:
   - Es el núcleo del chatbot. En un ciclo continuo, el chatbot espera la entrada del usuario, predice la intención de la frase, y responde en consecuencia.
   - Soporta varias intenciones como:
     - **Clima**: Pide al usuario una ciudad de Colombia y luego llama a la función `obtener_clima`.
     - **Noticias**: Pide un tema para buscar noticias relevantes mediante `obtener_noticias`.
     - **Nombre**: Responde con el nombre del chatbot.
     - **Agradecimiento**: Acepta las gracias del usuario con cortesía.
     - **Salir**: Termina la conversación cuando el usuario escribe 'salir' o 'quit'.
   - Las interacciones se guardan después de cada respuesta, registrando el diálogo.
   - Si el chatbot no puede interpretar la solicitud del usuario, se devuelve un mensaje solicitando una reformulación.

4. **Inicio del Chatbot**:
   - Finalmente, se ejecuta la función `chatbot_mejorado()` para iniciar la conversación con el usuario. El chatbot continúa interactuando hasta que el usuario decida terminar la sesión.


In [8]:
ciudades_colombia = {
    'bogota': {'lat': 4.7110, 'lon': -74.0721},
    'medellin': {'lat': 6.2442, 'lon': -75.5812},
    'cali': {'lat': 3.4516, 'lon': -76.53199},
    'barranquilla': {'lat': 10.9685, 'lon': -74.7813},
    'cartagena': {'lat': 10.3910, 'lon': -75.4794},
    'cucuta': {'lat': 7.8932, 'lon': -72.5012},
    'pasto': {'lat': 1.2132, 'lon': -77.2811},
    'manizales': {'lat': 5.0700, 'lon': -75.5174},
    'pereira': {'lat': 4.8133, 'lon': -75.6961},
    'ibague': {'lat': 4.4447, 'lon': -75.2424},
    'villavicencio': {'lat': 4.1448, 'lon': -73.6263},
    'bucaramanga': {'lat': 7.1254, 'lon': -73.1194},
    'armenia': {'lat': 4.5297, 'lon': -75.6811},
    'soledad': {'lat': 10.9833, 'lon': -74.7833},
    'sincelejo': {'lat': 9.3043, 'lon': -75.3921},
    'valledupar': {'lat': 10.4631, 'lon': -73.2537},
    'monteria': {'lat': 8.7550, 'lon': -75.8803},
    'neiva': {'lat': 2.9314, 'lon': -75.2803},
    'popayan': {'lat': 2.4479, 'lon': -76.6068},
    'envigado': {'lat': 6.1658, 'lon': -75.5867}
}

def obtener_clima(ciudad):
    api_key = os.getenv('OPENWEATHER_API_KEY')
    ciudad_lower = ciudad.lower().strip()

    if ciudad_lower in ciudades_colombia:
        lat = ciudades_colombia[ciudad_lower]['lat']
        lon = ciudades_colombia[ciudad_lower]['lon']
    else:
        return "No tengo información sobre esa ciudad. Por favor, verifica el nombre o elige una ciudad de Colombia que esté en la lista."

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=es"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if data.get("cod") == 200:
            temp = data['main']['temp']
            descripcion = data['weather'][0]['description']
            return f"El clima en {ciudad.title()} es {descripcion} con una temperatura de {temp}°C."
        else:
            return f"No pude encontrar el clima para {ciudad.title()}. Código de error: {data.get('cod')}, Mensaje: {data.get('message')}"

    except requests.exceptions.RequestException as err:
        return f"Error desconocido: {err}"

def obtener_noticias(tema):
    api_key = os.getenv('NEWS_API_KEY')
    url = f"https://newsapi.org/v2/everything?q={tema}&language=es&sortBy=popularity&apiKey={api_key}"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if data.get("status") == "ok":
            noticias = data['articles'][:5]
            resultado = []
            for noticia in noticias:
                titulo = textwrap.fill(f"Título: {noticia['title']}", width=70)
                descripcion = textwrap.fill(f"Descripción: {noticia.get('description', 'Descripción no disponible.')}", width=70)
                resultado.append(f"{titulo}\n{descripcion}")
            return "\n\n".join(resultado)
        else:
            return f"No pude obtener las noticias. Mensaje: {data.get('message')}"

    except requests.exceptions.RequestException as err:
        return f"Error desconocido: {err}"

def guardar_interaccion(usuario, bot):
    data = {"usuario": usuario, "bot": bot}
    with open('interacciones.json', 'a') as archivo:
        json.dump(data, archivo)
        archivo.write('\n')

def predecir_intencion(frase):
    frase = frase.lower().strip()

    if re.search(r'\b(clima|tiempo)\b', frase):
        return "clima"
    elif re.search(r'\b(noticias?|información|tema)\b', frase):
        return "noticias"
    elif re.search(r'\b(nombre|cómo te llamas)\b', frase):
        return "nombre"
    elif re.search(r'\b(gracias|agradezco)\b', frase):
        return "agradecer"
    elif re.search(r'\b(salir|adiós|quitar)\b', frase):
        return "salir"
    else:
        return None

def chatbot_mejorado():
    print("""
    ¡Hola! Soy un chatbot mejorado. Aquí están algunas cosas que puedes pedirme:
    - 'clima' o 'tiempo' para conocer el clima de una ciudad.
    - 'noticias' o 'información' para obtener noticias sobre un tema.
    - 'nombre' para saber cómo llamarme.
    - 'gracias' o 'agradezco' para agradecerme.
    - 'salir' o 'adiós' para finalizar la conversación.
    Escribe 'quit' para salir en cualquier momento.
    """)

    while True:
        usuario = input("Tú: ").lower().strip()

        if usuario == 'quit':
            print("Adiós por ahora. ¡Nos vemos pronto!")
            break

        intencion = predecir_intencion(usuario)

        if intencion == "clima":
            ciudad = input("¿En qué ciudad de Colombia te gustaría saber el clima? ").strip()
            respuesta = obtener_clima(ciudad)
        elif intencion == "noticias":
            tema = input("¿Sobre qué tema te gustaría leer noticias? ").strip()
            respuesta = obtener_noticias(tema)
        elif intencion == "nombre":
            respuesta = "Puedes llamarme 'bot'."
        elif intencion == "agradecer":
            respuesta = "¡De nada! Si tienes más preguntas, estaré aquí."
        elif intencion == "salir":
            print("Adiós por ahora. ¡Nos vemos pronto!")
            break
        else:
            respuesta = "No entendí eso. Por favor, intenta reformular tu solicitud."

        print(f"Chatbot:\n{respuesta}")

        guardar_interaccion(usuario, respuesta)

chatbot_mejorado()



    ¡Hola! Soy un chatbot mejorado. Aquí están algunas cosas que puedes pedirme:
    - 'clima' o 'tiempo' para conocer el clima de una ciudad.
    - 'noticias' o 'información' para obtener noticias sobre un tema.
    - 'nombre' para saber cómo llamarme.
    - 'gracias' o 'agradezco' para agradecerme.
    - 'salir' o 'adiós' para finalizar la conversación.
    Escribe 'quit' para salir en cualquier momento.
    
Tú: dame el clima en una ciudad
¿En qué ciudad de Colombia te gustaría saber el clima? pasto
Chatbot:
El clima en Pasto es lluvia ligera con una temperatura de 21.09°C.
Tú: quiero saber las noticias
¿Sobre qué tema te gustaría leer noticias? tesla
Chatbot:
Título: El Tesla Model Y Juniper dará la sorpresa: se filtra un cambio
totalmente inesperado
Descripción: Tesla estaría preparándose para darle una vuelta de
tuerca a su esperado Model Y Juniper. Según informan desde Reuters, la
división de la marca en China estaría planteándose la fabricación de
una nueva versión con un inte

Esta celda contiene una nueva mejora del **chatbot mejorado** que interactúa con el usuario y responde preguntas sobre el clima, noticias y realiza búsquedas en Google. El código se estructura de la siguiente manera:

1. **Función `buscar_en_google(query)`**:
   - Esta función realiza búsquedas en Google utilizando la **Google Custom Search API**. El usuario ingresa una consulta, y el chatbot devuelve los tres primeros resultados con el título, la descripción y el enlace correspondiente.
   - También se manejan los posibles errores de conexión o de la API.

2. **Función principal `chatbot_mejorado()`**:
   - La función principal del chatbot. Se ejecuta en un bucle continuo, donde el chatbot espera la entrada del usuario, predice la intención de su mensaje, y responde en consecuencia:
     - Si la intención es "clima", solicita una ciudad y llama a la función `obtener_clima`.
     - Si es "noticias", pide un tema y busca noticias relacionadas.
     - Si es "nombre", responde con el nombre del chatbot.
     - Si es "buscar" o "google", realiza una búsqueda en Google con el término que el usuario ingrese.
     - Si el usuario escribe "salir" o "quit", el chatbot finaliza la conversación.
   - Después de cada interacción, la respuesta se guarda en el archivo JSON.
   - Si la intención del usuario no se reconoce, el chatbot pide que reformule su solicitud.

8. **Ejecución del Chatbot**:
   - La función `chatbot_mejorado()` se ejecuta al final de la celda para iniciar la conversación con el usuario. El chatbot continuará funcionando hasta que el usuario decida salir escribiendo 'quit' o 'salir'.

Este chatbot combina el uso de varias **APIs externas** (OpenWeather, NewsAPI, Google Custom Search) para ofrecer una interacción útil y dinámica con el usuario, proporcionando información en tiempo real y guardando las conversaciones para referencia futura.

In [9]:
ciudades_colombia = {
    'bogota': {'lat': 4.7110, 'lon': -74.0721},
    'medellin': {'lat': 6.2442, 'lon': -75.5812},
    'cali': {'lat': 3.4516, 'lon': -76.53199},
    'barranquilla': {'lat': 10.9685, 'lon': -74.7813},
    'cartagena': {'lat': 10.3910, 'lon': -75.4794},
    'cucuta': {'lat': 7.8932, 'lon': -72.5012},
    'pasto': {'lat': 1.2132, 'lon': -77.2811},
    'manizales': {'lat': 5.0700, 'lon': -75.5174},
    'pereira': {'lat': 4.8133, 'lon': -75.6961},
    'ibague': {'lat': 4.4447, 'lon': -75.2424},
    'villavicencio': {'lat': 4.1448, 'lon': -73.6263},
    'bucaramanga': {'lat': 7.1254, 'lon': -73.1194},
    'armenia': {'lat': 4.5297, 'lon': -75.6811},
    'soledad': {'lat': 10.9833, 'lon': -74.7833},
    'sincelejo': {'lat': 9.3043, 'lon': -75.3921},
    'valledupar': {'lat': 10.4631, 'lon': -73.2537},
    'monteria': {'lat': 8.7550, 'lon': -75.8803},
    'neiva': {'lat': 2.9314, 'lon': -75.2803},
    'popayan': {'lat': 2.4479, 'lon': -76.6068},
    'envigado': {'lat': 6.1658, 'lon': -75.5867}
}

def obtener_clima(ciudad):
    api_key = os.getenv('OPENWEATHER_API_KEY')
    ciudad_lower = ciudad.lower().strip()

    if ciudad_lower in ciudades_colombia:
        lat = ciudades_colombia[ciudad_lower]['lat']
        lon = ciudades_colombia[ciudad_lower]['lon']
    else:
        return "No tengo información sobre esa ciudad. Por favor, verifica el nombre o elige una ciudad de Colombia que esté en la lista."

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=es"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if data.get("cod") == 200:
            temp = data['main']['temp']
            descripcion = data['weather'][0]['description']
            return f"El clima en {ciudad.title()} es {descripcion} con una temperatura de {temp}°C."
        else:
            return f"No pude encontrar el clima para {ciudad.title()}. Código de error: {data.get('cod')}, Mensaje: {data.get('message')}"

    except requests.exceptions.RequestException as err:
        return f"Error desconocido: {err}"

def obtener_noticias(tema):
    api_key = os.getenv('NEWS_API_KEY')
    url = f"https://newsapi.org/v2/everything?q={tema}&language=es&sortBy=popularity&apiKey={api_key}"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if data.get("status") == "ok":
            noticias = data['articles'][:5]
            resultado = []
            for noticia in noticias:
                titulo = textwrap.fill(f"Título: {noticia['title']}", width=70)
                descripcion = textwrap.fill(f"Descripción: {noticia.get('description', 'Descripción no disponible.')}", width=70)
                resultado.append(f"{titulo}\n{descripcion}")
            return "\n\n".join(resultado)
        else:
            return f"No pude obtener las noticias. Mensaje: {data.get('message')}"

    except requests.exceptions.RequestException as err:
        return f"Error desconocido: {err}"

def buscar_en_google(query):
    api_key = os.getenv('GOOGLE_API_KEY')
    cse_id = os.getenv('CSE_ID')
    url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={cse_id}&q={query}"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()

        if 'items' in data:
            resultados = data['items'][:3]
            resultado_formateado = []
            for result in resultados:
                titulo = result['title']
                link = result['link']
                snippet = result.get('snippet', 'No hay descripción.')
                resultado_formateado.append(f"Título: {titulo}\nDescripción: {snippet}\nEnlace: {link}\n")
            return "\n".join(resultado_formateado)
        else:
            return "No encontré resultados relevantes para tu búsqueda."

    except requests.exceptions.RequestException as err:
        return f"Error realizando la búsqueda: {err}"

def guardar_interaccion(usuario, bot):
    data = {"usuario": usuario, "bot": bot}
    with open('interacciones.json', 'a') as archivo:
        json.dump(data, archivo)
        archivo.write('\n')

def predecir_intencion(frase):
    frase = frase.lower().strip()

    if re.search(r'\b(clima|tiempo)\b', frase):
        return "clima"
    elif re.search(r'\b(noticias?|información|tema)\b', frase):
        return "noticias"
    elif re.search(r'\b(nombre|cómo te llamas)\b', frase):
        return "nombre"
    elif re.search(r'\b(gracias|agradezco)\b', frase):
        return "agradecer"
    elif re.search(r'\b(buscar|google)\b', frase):
        return "google"
    elif re.search(r'\b(salir|adiós|quitar)\b', frase):
        return "salir"
    else:
        return None

def chatbot_mejorado():
    print("""
    ¡Hola! Soy un chatbot mejorado. Aquí están algunas cosas que puedes pedirme:
    - 'clima' o 'tiempo' para conocer el clima de una ciudad.
    - 'noticias' o 'información' para obtener noticias sobre un tema.
    - 'nombre' para saber cómo llamarme.
    - 'buscar' o 'google' para realizar una búsqueda en Google.
    - 'gracias' o 'agradezco' para agradecerme.
    - 'salir' o 'adiós' para finalizar la conversación.
    Escribe 'quit' para salir en cualquier momento.
    """)

    while True:
        usuario = input("Tú: ").lower().strip()

        if usuario == 'quit':
            print("Adiós por ahora. ¡Nos vemos pronto!")
            break

        intencion = predecir_intencion(usuario)

        if intencion == "clima":
            ciudad = input("¿En qué ciudad de Colombia te gustaría saber el clima? ").strip()
            respuesta = obtener_clima(ciudad)
        elif intencion == "noticias":
            tema = input("¿Sobre qué tema te gustaría leer noticias? ").strip()
            respuesta = obtener_noticias(tema)
        elif intencion == "nombre":
            respuesta = "Puedes llamarme 'bot'."
        elif intencion == "google":
            query = input("¿Qué te gustaría buscar en Google? ").strip()
            respuesta = buscar_en_google(query)
        elif intencion == "agradecer":
            respuesta = "¡De nada! Si tienes más preguntas, estaré aquí."
        elif intencion == "salir":
            print("Adiós por ahora. ¡Nos vemos pronto!")
            break
        else:
            respuesta = "No entendí eso. Por favor, intenta reformular tu solicitud."

        print(f"Chatbot:\n{respuesta}")

        guardar_interaccion(usuario, respuesta)

chatbot_mejorado()


    ¡Hola! Soy un chatbot mejorado. Aquí están algunas cosas que puedes pedirme:
    - 'clima' o 'tiempo' para conocer el clima de una ciudad.
    - 'noticias' o 'información' para obtener noticias sobre un tema.
    - 'nombre' para saber cómo llamarme.
    - 'buscar' o 'google' para realizar una búsqueda en Google.
    - 'gracias' o 'agradezco' para agradecerme.
    - 'salir' o 'adiós' para finalizar la conversación.
    Escribe 'quit' para salir en cualquier momento.
    
Tú: dame el clima para una ciudad
¿En qué ciudad de Colombia te gustaría saber el clima? bogota
Chatbot:
El clima en Bogota es muy nuboso con una temperatura de 16.99°C.
Tú: dame las principales noticias 
¿Sobre qué tema te gustaría leer noticias? facebook
Chatbot:
Título: El gobierno de EE. UU. presionó a Facebook para censurar
publicaciones de la COVID-19
Descripción: Mark Zuckerberg declaró que el gobierno de Estados Unidos
presionó a Facebook para censurar publicaciones de la COVID-19. El
director ejecutivo de 